In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os

In [2]:
with open('oneliners_data.txt', 'r') as f:
    text = f.read()

In [3]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [5]:
def one_hot_encode(arr, n_labels):
    
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
def get_batches(arr, batch_size, seq_length):

    
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y 


In [8]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [10]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [11]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=612, n_layers=4,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):

        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        

        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
       
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):

        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [12]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):

    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()


            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() 
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [15]:
n_hidden=512
n_layers=4

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(96, 512, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=96, bias=True)
)


In [16]:
batch_size = 128
seq_length = 160 
n_epochs = 100 

train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/100... Step: 10... Loss: 3.3051... Val Loss: 3.2582
Epoch: 1/100... Step: 20... Loss: 3.2393... Val Loss: 3.2207
Epoch: 1/100... Step: 30... Loss: 3.2222... Val Loss: 3.2116
Epoch: 1/100... Step: 40... Loss: 3.2135... Val Loss: 3.2084
Epoch: 1/100... Step: 50... Loss: 3.2100... Val Loss: 3.2078
Epoch: 1/100... Step: 60... Loss: 3.1984... Val Loss: 3.2079
Epoch: 1/100... Step: 70... Loss: 3.2106... Val Loss: 3.2076
Epoch: 1/100... Step: 80... Loss: 3.1767... Val Loss: 3.2070
Epoch: 2/100... Step: 90... Loss: 3.1945... Val Loss: 3.2065
Epoch: 2/100... Step: 100... Loss: 3.1984... Val Loss: 3.2066
Epoch: 2/100... Step: 110... Loss: 3.1873... Val Loss: 3.2068
Epoch: 2/100... Step: 120... Loss: 3.2181... Val Loss: 3.2068
Epoch: 2/100... Step: 130... Loss: 3.1803... Val Loss: 3.2071
Epoch: 2/100... Step: 140... Loss: 3.2000... Val Loss: 3.2067
Epoch: 2/100... Step: 150... Loss: 3.1893... Val Loss: 3.2064
Epoch: 2/100... Step: 160... Loss: 3.1924... Val Loss: 3.2067
Epoch: 3/100... S

In [17]:
model_dante = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_dante, 'wb') as f:
    torch.save(checkpoint, f)

In [18]:
def predict(net, char, h=None, top_k=None):

        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = net(inputs, h)

        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() 

        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return net.int2char[char], h

In [19]:
def sample(net, size, prime='This', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() 
    
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [20]:
print(sample(net, 10000, prime='<S> ', top_k=5))

<S> What do you still be considered short from my paint? <E> <S> I have a gardance of a couration attach is well because I decided to go to a champoon <E> <S> I have a party to a brain taker, see about his finger. <E> <S> There are no more important, but I can buy a billion penny. <E> <S> When my wallet should be attending the shaming stop that we hope my foot said "I don't know what me." <E> <S> I will see the worst minute a shitty pancake for some positive some of that book is the best way to could be saying that I can't get important as a bad poor. <E> <S> My friend told me to see my dreams of my back at single that I can't colorbly to a lot. <E> <S> The person who works, it would be taking the way your back taste on their panes <E> <S> A pedophile talks is a crowd of a little too stack to me. <E> <S> The opposite of a movie always been so many school all dog <E> <S> It was an all of the postman that my shampoo is the worst salad thing, there are a practice or them. <E> <S> I'm a so

In [21]:
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [22]:
print(sample(loaded, 10000, top_k=5, prime="<S> "))

<S> I heard they're so political shaped and says in the can <E> <S> A man who don't take away with the world when you should see myself in the second. <E> <S> I think my girlfriend was too lot of and started to be so so trainers as a still, I think I went on her off an electricist. <E> <S> We all deal a book about the propesty of my wife. <E> <S> Whoever was the wailer, I was throwing me on my friends but now, they would be caught it. <E> <S> I wondered why the break and all the movie was a poor positive that was a short fall. <E> <S> I would spell short tennis, I'm starting to be anything alreidene. <E> <S> If an and person walks into a ball and thought I had how his water and the witch is when you worked to myself. <E> <S> I was going to tell you my grandfather, but it's been a bear but it was some proting on at them. <E> <S> To me sure, whats: as I saw, is a complete, bears the produster <E> <S> I take shoes and now the word answer to sell me her work, the brain is an alphabet. <E> 